In [18]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from parsel import Selector
from selenium.webdriver.common.keys import Keys
from facebook_scraper import get_posts ,get_posts_by_search
from multiprocessing import Pool, cpu_count
import re
from requests import get
import pandas as pd
import numpy as np
import time
import os
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
from datetime import datetime

########################################### Debut du traitement ##############################################

In [9]:
def typeMaison(publication,bag_app,bag_mais):
     dicoType={}
     count=1
     for word in publication:
        if word in bag_app:
            dicoType[count]='appartement'
            count=count+1  
        elif word in bag_mais:
            dicoType[count]='maison Individuel'
            count=count+1  
     return dicoType

In [8]:

def check(post_elmt):
    post_elmt.replace(".","")
    post_elmt = re.findall("\d+", post_elmt)
    if len(post_elmt) != 0 :
        post_elmt = post_elmt[0]  
        return post_elmt
    return 0

def montant_loyer_piece(sac_mot, post):
    dico_montant = {}
    post_suiv = ""
    cpt=0
    for i in range(0,len(post)):
        loyer=0
        if post[i] in sac_mot:
            # break
            post_pred = post[i-1]
            if i != (len(post)-1):
                post_suiv = post[i+1]
            loyer = check(post_pred)
            if loyer!=0:
                    cpt+=1
                    dico_montant[cpt]= loyer
            if i != (len(post)-1):        
                if ((loyer == 0) and (i != (len(post)-2))):
                    loyer = check(post[i+1])
                    if loyer!=0:
                        cpt+=1
                        dico_montant[cpt]= loyer
            
        if loyer==0:
            val=re.findall("[a-zA-Z]+", post[i])
            if len(val)!=0:
                if val[0].upper() in sac_mot:
                    loyer= check(post[i])
                    if loyer!=0:
                        cpt=cpt+1
                        dico_montant[cpt]= loyer

    return dico_montant


In [10]:
def standing(sac,eco,haut,moyen):
    dicoStanding = dict()
    k = 1
    for i in sac:
        if i in eco:
            dicoStanding[k] = "econonmique"
            k += 1
            continue
        if i in haut:
            dicoStanding[k] = "haut"
            k += 1
            continue
        if i in moyen:
            dicoStanding[k] = "moyen"
            k += 1     
    return dicoStanding

In [11]:
immeuble_sac_mot =['REZ','APPARTEMENT','BALCON','APPARTEMENTS','APARTEMENT','APARTEMENTS', 
           'APPATEMENT','APPATEMENTS', 'APPAT', 'APPATS','IMMEUBLE',
           'IMMEUBLES','IMEUBLE','IMMEBLE',  'IMMEBLES','IMEBLE', 'IMEBLES','1ER','2EM','ETAGE']
villa_sac_mot=["VILLA","BASSE","VILA",'VILLAS','VILAS']
piece_sac_mot=["PIECE","PIECES","PIEC",'PCS']
montant_loyer_sac_mot = ["PRIX","LOYER", "FCFA","F","CFA","FR","MIL","MILLE","MILLES",'XOF','FA','€','$']
motCleLocationValide=["LOYER","LOUER","LOUE","LOU","LOCATION","LOCASSION","LOCASION","VISITE","CAUTION","CAUSION","CAUSSION","COSION","COSSION"]
motCleLocationExclu=["VENTE","VENT","TERRAIN","TERRIN","TERAIN","FONCIER","ACHETER","ACHETE"]

In [12]:
from unidecode import unidecode
import re
import codecs
def nettoyage(compltexte):
    compltexte=str(compltexte)
    compltexte = compltexte.replace("*", "")
    compltexte = compltexte.replace(",", "")
    compltexte = compltexte.replace(".", "")
    compltexte = re.sub(r"[^\w\s]", " ", compltexte) # supprime les caractères spéciaux et symboles
    listMot = compltexte.split() 
    listMot = [mot.upper() for mot in listMot]
    listMot = [unidecode(mot) for mot in listMot]
    return listMot

    


In [13]:

def first(postText):
    listTraite = nettoyage(postText)
    val=[]
    for mot in listTraite:
            if mot in motCleLocationValide:
                val= listTraite                
            elif mot in motCleLocationExclu:
                val=  []
            else:
                val= listTraite
    return val

        

BINGERVILLE

In [22]:
import pandas as pd
database = pd.read_csv("D:/Master2_2022-2023/WEBSCRAPING/WEB_SCRAPING_PROJECT/data/bingerville.csv")
def traitement(database,localite):
    format = '%Y-%m-%d %H:%M:%S'
    baseFinal =[]
    for ligne in range(database.shape[0]):
        nuplet = database.iloc[ligne,:]
        publicationOk=first(nuplet["post_text"])
        if len(publicationOk) !=0:    
            typeM= typeMaison(publicationOk,immeuble_sac_mot,villa_sac_mot)
            piece = montant_loyer_piece(piece_sac_mot,publicationOk)
            montant = montant_loyer_piece(montant_loyer_sac_mot,publicationOk)
            if len(typeM)==1 and len(piece)==1 and len(montant)==1:
                ligneAEcricre = {}
                ligneAEcricre["username"] = nuplet["username"]
                ligneAEcricre["post_url"] = nuplet["post_url"]
                temps = datetime.strptime(str(nuplet["time"]),format)
                ligneAEcricre["jour"] = temps.day
                ligneAEcricre["imageMaisonLien"] = nuplet["images_lowquality"]
                ligneAEcricre["mois"] = temps.month
                ligneAEcricre["annee"] = temps.year
                ligneAEcricre["localite"] = localite
                ligneAEcricre["type"] = list(typeM.values())[0]
                ligneAEcricre["montant"] = list(montant.values())[0]
                ligneAEcricre["nbPiece"] = list(piece.values())[0]
                baseFinal.append(ligneAEcricre)
            if len(typeM)==0 and len(piece)==1 and len(montant)==1:
                ligneAEcricre = {}
                ligneAEcricre["username"] = nuplet["username"]
                ligneAEcricre["post_url"] = nuplet["post_url"]
                temps = datetime.strptime(str(nuplet["time"]),format)
                ligneAEcricre["jour"] = temps.day
                ligneAEcricre["mois"] = temps.month
                ligneAEcricre["annee"] = temps.year
                ligneAEcricre["imageMaisonLien"] = nuplet["images_lowquality"]
                ligneAEcricre["localite"] = localite
                ligneAEcricre["type"] = ""
                ligneAEcricre["montant"] =list( montant.values())[0]
                ligneAEcricre["nbPiece"] = list(piece.values())[0]
                baseFinal.append(ligneAEcricre)
            if len(typeM)>1 and len(piece)>1 and len(montant)>1:
                for ii in range(min([len(typeM),len(piece),len(montant)])):
                    ligneAEcricre = {}
                    ligneAEcricre["username"] = nuplet["username"]
                    ligneAEcricre["post_url"] = nuplet["post_url"]
                    temps = datetime.strptime(str(nuplet["time"]),format)
                    ligneAEcricre["jour"] = temps.day
                    ligneAEcricre["mois"] = temps.month
                    ligneAEcricre["annee"] = temps.year
                    ligneAEcricre["imageMaisonLien"] = nuplet["images_lowquality"]
                    ligneAEcricre["localite"] = localite
                    ligneAEcricre["type"] = list(typeM.values())[ii]
                    ligneAEcricre["montant"] = list(montant.values())[ii]
                    ligneAEcricre["type"] = list(typeM.values())[ii]
                    baseFinal.append(ligneAEcricre) 
        
    return pd.DataFrame(baseFinal).to_csv("Base_Final"+localite+".csv")      


In [23]:
# Avant traitement de la base
database.head()

,Unnamed: 0,post_id,text,post_text,shared_text,original_text,time,timestamp,image,image_lowquality,...,with,page_id,sharers,image_id,image_ids,video_ids,videos,was_live,header,fetched_time
0,0,1836112673423701,Très bon studio 60 000 x 5 au carrefour châtea...,Très bon studio 60 000 x 5 au carrefour châtea...,NaN,NaN,2023-01-19 13:06:01,1.674134e+09,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,...,"[{'name': 'Pour toute maison à BINGERVILLE', '...",1443698102665162,NaN,NaN,[],[],[],False,Romeo Jésus Gnahore‎Pour toute maison à BINGER...,2023-02-19 03:42:33.178032
1,1,1823766881324947,Grand studio américain avec terrasse arrière à...,Grand studio américain avec terrasse arrière à...,NaN,NaN,2023-01-02 23:09:25,1.672701e+09,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,...,"[{'name': 'Pour toute maison à BINGERVILLE', '...",1443698102665162,NaN,NaN,[],[],[],False,MKouassi Mathieu‎Pour toute maison à BINGERVILLE,2023-02-19 03:42:39.011272
2,2,1839673466400955,Studio 60000f disponible à bingerville\nNuméro...,Studio 60000f disponible à bingerville\nNuméro...,NaN,NaN,2023-01-24 08:02:49,1.674547e+09,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,...,"[{'name': 'Bingerville, Abidjan', 'link': '/pr...",103142169726270,NaN,NaN,[],[],[],False,"Christian Morel Diby Koffi est à Bingerville, ...",2023-02-19 03:42:45.034628
3,3,1854971384871163,2pieces 80mille disponibles à bingerville avec...,2pieces 80mille disponibles à bingerville avec...,NaN,NaN,2023-02-13 07:48:07,1.676274e+09,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,...,"[{'name': 'Pour toute maison à BINGERVILLE', '...",1443698102665162,NaN,NaN,[],[],[],False,Bertrand Djaha Narrow‎Pour toute maison à BING...,2023-02-19 03:42:50.950639
4,4,1850601348641500,Studio Américain individuel avec Petit Salon 6...,Studio Américain individuel avec Petit Salon 6...,NaN,NaN,2023-02-07 14:19:49,1.675780e+09,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,https://scontent.fabj4-2.fna.fbcdn.net/v/t39.3...,...,"[{'name': 'Pour toute maison à BINGERVILLE', '...",1443698102665162,NaN,NaN,[],[],[],False,Jules Konan‎Pour toute maison à BINGERVILLE,2023-02-19 03:43:00.500694


In [24]:
# Nombre de ligne avant traitement
len(database)

363

In [25]:
traitement(database,"Bingerville")

In [26]:
pd.read_csv("Base_FinalBingerville.csv")


,Unnamed: 0,username,post_url,jour,mois,annee,imageMaisonLien,localite,type,montant,nbPiece
0,0,Bertrand Djaha Narrow,https://facebook.com/1443698102665162/posts/18...,13,2,2023,['https://scontent.fabj4-2.fna.fbcdn.net/v/t39...,Bingerville,NaN,80,2.0
1,1,Guy-Ange Kff Kss,https://facebook.com/1443698102665162/posts/18...,13,2,2023,['https://scontent.fabj4-2.fna.fbcdn.net/v/t39...,Bingerville,NaN,200,3.0
2,2,Serge Kena,https://facebook.com/1443698102665162/posts/18...,13,2,2023,[],Bingerville,maison Individuel,150000,3.0
3,3,Bertrand Djaha Narrow,https://facebook.com/1443698102665162/posts/18...,13,2,2023,['https://scontent.fabj4-2.fna.fbcdn.net/v/t39...,Bingerville,NaN,80,2.0
4,4,Guy-Ange Kff Kss,https://facebook.com/1443698102665162/posts/18...,13,2,2023,['https://scontent.fabj4-2.fna.fbcdn.net/v/t39...,Bingerville,NaN,200,3.0
5,5,Serge Kena,https://facebook.com/1443698102665162/posts/18...,13,2,2023,[],Bingerville,maison Individuel,150000,3.0
6,6,Trouve ta maison / appartement à Genas,https://facebook.com/trouvetamaison/posts/3777...,26,8,2022,['https://external.fabj3-2.fna.fbcdn.net/emg1/...,Bingerville,NaN,325000,3.0
7,7,Saturnin Trayé,https://facebook.com/210185212377761/posts/608...,18,2,2023,['https://scontent.fabj3-2.fna.fbcdn.net/v/t39...,Bingerville,NaN,40,4.0
8,8,Mike Despe,https://facebook.com/210185212377761/posts/608...,18,2,2023,['https://scontent.fabj3-2.fna.fbcdn.net/v/t39...,Bingerville,appartement,500000,NaN
9,9,Mike Despe,https://facebook.com/210185212377761/posts/608...,18,2,2023,['https://scontent.fabj3-2.fna.fbcdn.net/v/t39...,Bingerville,appartement,500000,NaN


In [27]:
len(pd.read_csv("Base_FinalBingerville.csv"))

29

FAYA

In [62]:
import pandas as pd
database1 = pd.read_csv("D:/Master2_2022-2023/WEBSCRAPING/WEB_SCRAPING_PROJECT/data/Faya.csv")
def traitement(database,localite):
    format = '%Y-%m-%d %H:%M:%S'
    baseFinal =[]
    for ligne in range(database.shape[0]):
        nuplet = database.iloc[ligne,:]
        publicationOk=first(nuplet["post_text"])
        if len(publicationOk) !=0:    
            typeM= typeMaison(publicationOk,immeuble_sac_mot,villa_sac_mot)
            piece = montant_loyer_piece(piece_sac_mot,publicationOk)
            montant = montant_loyer_piece(montant_loyer_sac_mot,publicationOk)
            if len(typeM)==1 and len(piece)==1 and len(montant)==1:
                ligneAEcricre = {}
                ligneAEcricre["username"] = nuplet["username"]
                ligneAEcricre["post_url"] = nuplet["post_url"]
                temps = datetime.strptime(str(nuplet["time"]),format)
                ligneAEcricre["jour"] = temps.day
                ligneAEcricre["imageMaisonLien"] = nuplet["images_lowquality"]
                ligneAEcricre["mois"] = temps.month
                ligneAEcricre["annee"] = temps.year
                ligneAEcricre["localite"] = localite
                ligneAEcricre["type"] = list(typeM.values())[0]
                ligneAEcricre["montant"] = list(montant.values())[0]
                ligneAEcricre["nbPiece"] = list(piece.values())[0]
                baseFinal.append(ligneAEcricre)
            if len(typeM)==0 and len(piece)==1 and len(montant)==1:
                ligneAEcricre = {}
                ligneAEcricre["username"] = nuplet["username"]
                ligneAEcricre["post_url"] = nuplet["post_url"]
                temps = datetime.strptime(str(nuplet["time"]),format)
                ligneAEcricre["jour"] = temps.day
                ligneAEcricre["mois"] = temps.month
                ligneAEcricre["annee"] = temps.year
                ligneAEcricre["imageMaisonLien"] = nuplet["images_lowquality"]
                ligneAEcricre["localite"] = localite
                ligneAEcricre["type"] = ""
                ligneAEcricre["montant"] =list( montant.values())[0]
                ligneAEcricre["nbPiece"] = list(piece.values())[0]
                baseFinal.append(ligneAEcricre)
            if len(typeM)>1 and len(piece)>1 and len(montant)>1:
                for ii in range(min([len(typeM),len(piece),len(montant)])):
                    ligneAEcricre = {}
                    ligneAEcricre["username"] = nuplet["username"]
                    ligneAEcricre["post_url"] = nuplet["post_url"]
                    temps = datetime.strptime(str(nuplet["time"]),format)
                    ligneAEcricre["jour"] = temps.day
                    ligneAEcricre["mois"] = temps.month
                    ligneAEcricre["annee"] = temps.year
                    ligneAEcricre["imageMaisonLien"] = nuplet["images_lowquality"]
                    ligneAEcricre["localite"] = localite
                    ligneAEcricre["type"] = list(typeM.values())[ii]
                    ligneAEcricre["montant"] = list(montant.values())[ii]
                    ligneAEcricre["type"] = list(typeM.values())[ii]
                    baseFinal.append(ligneAEcricre) 
        
    return pd.DataFrame(baseFinal).to_csv("Base_Final"+localite+".csv")

In [63]:
# Avant traitement 
database1.head()

,Unnamed: 0,post_id,text,post_text,shared_text,original_text,time,timestamp,image,image_lowquality,...,image_id,image_ids,was_live,header,fetched_time,video_ids,videos,listing_title,listing_price,listing_location
0,0,3126337780998669,Bonjour chers membres de ce groupe\nJe voudrai...,Bonjour chers membres de ce groupe\nJe voudrai...,NaN,NaN,2023-01-03 12:41:24,1.672750e+09,NaN,https://scontent.fabj3-2.fna.fbcdn.net/m1/v/t6...,...,NaN,[],False,Alex Kenley Watson‎MAISON A LOUER A ABOBO,2023-02-19 01:35:21.022076,NaN,NaN,NaN,NaN,NaN
1,1,3160722087560238,Bonjour appareil photo en brade 65k intéressé ...,Bonjour appareil photo en brade 65k intéressé ...,NaN,NaN,2023-02-19 01:28:02,1.676770e+09,https://scontent.fabj3-2.fna.fbcdn.net/v/t39.3...,https://scontent.fabj3-2.fna.fbcdn.net/v/t39.3...,...,NaN,[],False,Kere Zeinab Yasmine‎MAISON A LOUER A ABOBO,2023-02-19 01:35:25.393908,[],[],NaN,NaN,NaN
2,2,3160720504227063,"💋💋💋Dramatic, Hot💋💋💋\n\n💋💋💋Dramatique, chaud💋💋💋","💋💋💋Dramatic, Hot💋💋💋\n\n💋💋💋Dramatique, chaud💋💋💋",NaN,"💋💋💋Dramatic, Hot💋💋💋",2023-02-19 01:25:19,1.676770e+09,NaN,https://scontent.fabj3-2.fna.fbcdn.net/v/t39.3...,...,NaN,[],False,Steff Tivoli‎MAISON A LOUER A ABOBO,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3160708534228260,Chambre salon 50.000 abobo ndotre kobakro au 2...,Chambre salon 50.000 abobo ndotre kobakro au 2...,NaN,NaN,2023-02-19 01:03:08,1.676769e+09,https://scontent.fabj3-2.fna.fbcdn.net/v/t39.3...,https://scontent.fabj3-2.fna.fbcdn.net/v/t39.3...,...,NaN,[],False,Jm Le Riche‎MAISON A LOUER A ABOBO,2023-02-19 01:35:30.953766,[],[],NaN,NaN,NaN
4,4,3160698134229300,studio américain 80.000 ×5.\nTrès propre\nA an...,studio américain 80.000 ×5.\nTrès propre\nA an...,NaN,NaN,2023-02-19 00:46:00,1.676768e+09,NaN,https://scontent.fabj3-2.fna.fbcdn.net/m1/v/t6...,...,NaN,[],False,Jérôme Korendji‎MAISON A LOUER A ABOBO,2023-02-19 01:35:31.611279,NaN,NaN,NaN,NaN,NaN


In [64]:
# taille avant traitement 
len(database1)

189

In [65]:
traitement(database1,"Faya")

In [66]:
pd.read_csv("Base_FinalFaya.csv")

,Unnamed: 0,username,post_url,jour,imageMaisonLien,mois,annee,localite,type,montant,nbPiece
0,0,Chalay Immobilier Et Service,https://facebook.com/2363867107241571/posts/29...,14,['https://scontent.fabj4-2.fna.fbcdn.net/v/t39...,2,2023,Faya,appartement,160,3.0
1,1,Youssef Pokou,https://facebook.com/122946726348599/posts/660...,18,['https://scontent.fabj4-2.fna.fbcdn.net/v/t39...,2,2023,Faya,NaN,1,4.0
2,2,Legéant immobilier,https://facebook.com/122946726348599/posts/661...,18,['https://scontent.fabj4-2.fna.fbcdn.net/v/t39...,2,2023,Faya,NaN,450,6.0
3,3,Wilfried Anderson Gbazale,https://facebook.com/122946726348599/posts/657...,14,['https://scontent.fabj4-2.fna.fbcdn.net/v/t39...,2,2023,Faya,maison Individuel,65,NaN
4,4,Wilfried Anderson Gbazale,https://facebook.com/122946726348599/posts/657...,14,['https://scontent.fabj4-2.fna.fbcdn.net/v/t39...,2,2023,Faya,maison Individuel,65,NaN
...,...,...,...,...,...,...,...,...,...,...,...
84,84,Wilfried Anderson Gbazale,https://facebook.com/122946726348599/posts/657...,14,['https://scontent.fabj3-2.fna.fbcdn.net/v/t39...,2,2023,Faya,maison Individuel,200,NaN
85,85,Wilfried Anderson Gbazale,https://facebook.com/122946726348599/posts/657...,14,['https://scontent.fabj3-2.fna.fbcdn.net/v/t39...,2,2023,Faya,appartement,150,NaN
86,86,Wilfried Anderson Gbazale,https://facebook.com/122946726348599/posts/657...,14,['https://scontent.fabj3-2.fna.fbcdn.net/v/t39...,2,2023,Faya,maison Individuel,140,NaN
87,87,Wilfried Anderson Gbazale,https://facebook.com/122946726348599/posts/657...,14,['https://scontent.fabj3-2.fna.fbcdn.net/v/t39...,2,2023,Faya,maison Individuel,130,NaN


In [67]:
len(pd.read_csv("Base_FinalFaya.csv"))

89